In [1]:
%pip install openpyxl

%pip install xlrd

import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
file_name = "T2-24-25-Enrollments-after-Add-Drop-Process-v3.xls"
try:
    df = pd.read_excel(file_name, sheet_name="Course Roster Report", usecols=["Course","Section","OTHER/REGULAR-ELECTIVE/REGULAR-CORE","Course Variant","Student ID","Email ID","Batch"])
    print("Data successfully loaded into DataFrame.")
    df.to_csv("enrollment_master.csv", index=False)
    print("Filtered data successfully saved to 'enrollment_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'enrollment_master.csv'.


In [3]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
# df2 = pd.read_csv(file_name)
df1 =  pd.DataFrame()

df = df[df['OTHER/REGULAR-ELECTIVE/REGULAR-CORE'] != 'OTHER']
df1['Code'] = df['Course'].str.split('/').str[0]
df1['Title'] = df['Course'].str.split('/').str[1]
df['CourseCode'] = df['Course'].str.split('/').str[0]
df['CourseTitle'] = df['Course'].str.split('/').str[1]

df1 = df1.drop_duplicates()
df = df.drop_duplicates()

df.drop('Course', axis=1, inplace=True)

df1.to_csv("course_master.csv", index=False)
df.to_csv("enrollment_master.csv", index=False)


In [4]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
new_order = [
    'CourseCode',
    'CourseTitle',
    'OTHER/REGULAR-ELECTIVE/REGULAR-CORE',
    'Course Variant',
    'Student ID',
    'Batch',
    'Section',
    'Email ID'
]

df = df[new_order]

# rename column Email ID to EmailID

df.rename(columns={'Email ID': 'EmailID'}, inplace=True)

df.to_csv(file_name, index=False)

In [5]:
# Add a new column called Lab in course_master.csv if 'lab' string is found in the title column

file_name = "course_master.csv"
df = pd.read_csv(file_name)

df['Lab'] = df['Title'].str.contains('lab', case=False)
df.to_csv(file_name, index=False)

In [6]:
# Add a new column called Lab in classroom_master.csv if 'LAB' string is found in the RoomNumber column

file_name = "classroom_master.csv"
df = pd.read_csv(file_name)

df['Lab'] = df['RoomNumber'].str.contains('LAB', case=False)
df.to_csv(file_name, index=False)



In [7]:
# Add a new column called Dept in course_master.csv if 'CSE ' string or 'AIM' or 'DAS' is found in the Code column then set the value as CSE else set it as ECE

file_name = "course_master.csv"

df = pd.read_csv(file_name)

df['Dept'] = df['Code'].str.contains('CSE|AIM|DAS', case=False)
df['Dept'] = df['Dept'].map({True: 'CSE', False: 'ECE'})
df.to_csv(file_name, index=False)


In [8]:
# Add a new column called DeptLab in classroom_master.csv if Lab column is True and 'R' string is found in the RoomNumber column

file_name = "classroom_master.csv"
df = pd.read_csv(file_name)

df['DeptLab'] = df['Lab'] & df['RoomNumber'].str.contains('R', case=False)
df.to_csv(file_name, index=False)

In [9]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)

# "CSE 102 P , Data Structures and Algorithms lab,REGULAR-CORE,CSE 102 P/Data Structures and Algorithms lab/Term II 2024-25/T224-CSE 102P-Sec-A,BT2024001,2024-25 B.Tech CSE,T224-CSE 102P-Sec-A,Arjun.Sai@iiitb.ac.in"
# This is an entry in enrollment.csv file 
# I want to add update the existing 'Section' column. If the current 'Section' columnn value is 'T224-CSE 102P-Sec-A' then I want to update it to 'A' and if it is 'T224-CSE 102P-Sec-B' then I want to update it to 'B' and if there is no section mentioned then just change it to 'A'
# do it using a for loop
#It is just an exmaple 
# for index, row in df.iterrows():
#     if row['Section'] == 'T224-CSE 102P-Sec-A':
#         df.at[index, 'Section'] = 'A'
#     elif row['Section'] == 'T224-CSE 102P-Sec-B':
#         df.at[index, 'Section'] = 'B'
#     else:
#         df.at[index, 'Section'] = 'A'

for index,row in df.iterrows():
    if 'Sec' in row['Section']:
        df.at[index, 'Section'] = row['Section'].split('-')[-1]
    else:
        df.at[index, 'Section'] = 'A'

df.to_csv(file_name, index=False)

In [10]:
# # Create a new file with columns as Code and Exam Choice. The entries for column Code are same as from 'course_master.csv' and fill 'Same' for everythig in Exam choice column

# file_name = "course_master.csv"
# df = pd.read_csv(file_name)

# df1 = pd.DataFrame()
# df1['Code'] = df['Code']
# df1['Exam Choice'] = 'Same'
# df1.to_csv("exam_choice.csv", index=False)

In [11]:
# Delete entries in 'enrollment_master.csv' file where the coursecode matches with code column in exam_choice.csv but the exam choice column is 'NoExam'ArithmeticError

df = pd.read_csv("enrollment_master.csv")

df1 = pd.read_csv("exam_choice.csv")

for index1, row1 in df1.iterrows():
    if df1.at[index1, 'Exam Choice'] == 'NoExam':
        # loop through all the entries of df and check if the course code matches with the code in df1 and the section is 'A' then update the CourseCode to 'CSE 102 P-A'
        for index, row in df.iterrows():
            if row['CourseCode'] == row1['Code']:
                df.drop(index, inplace=True)

df.to_csv("enrollment_master.csv", index=False)

In [12]:
# Update column Coursecode in 'enrollment_master.csv' file. If the value is 'CSE 102 P' then update it to 'CSE 102 P-A' if the section is 'A' and the the corresponding entry in exam_choice.csv is different

df = pd.read_csv("enrollment_master.csv")

df1 = pd.read_csv("exam_choice.csv")

for index1, row1 in df1.iterrows():
    if df1.at[index1, 'Exam Choice'] != 'Same':
        # print(row1['Code'])
        # df.at[index, 'CourseCode'] = row['CourseCode'] + '-' + row['Section']
        # loop through all the entries of df and check if the course code matches with the code in df1 and the section is 'A' then update the CourseCode to 'CSE 102 P-A'
        for index, row in df.iterrows():
            if row['CourseCode'] == row1['Code']:
                # print(row['CourseCode'])
                df.at[index, 'CourseCode'] = row['CourseCode'] + '-' + row['Section']
                # print(row['CourseCode'])

df.to_csv("enrollment_master.csv", index=False)


In [13]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
# df2 = pd.read_csv(file_name)
df1 =  pd.DataFrame()

df1['Code'] = df['CourseCode']
df1['Title'] = df['CourseTitle'] 

df1 = df1.drop_duplicates()
df1.to_csv("course_master.csv", index=False)


In [14]:
# Add a new column called Lab in course_master.csv if 'lab' string is found in the title column

file_name = "course_master.csv"
df = pd.read_csv(file_name)

df['Lab'] = df['Title'].str.contains('lab', case=False)
df.to_csv(file_name, index=False)

In [15]:
# Add a new column called Dept in course_master.csv if 'CSE ' string or 'AIM' or 'DAS' is found in the Code column then set the value as CSE else set it as ECE

file_name = "course_master.csv"

df = pd.read_csv(file_name)

df['Dept'] = df['Code'].str.contains('CSE|AIM|DAS', case=False)
df['Dept'] = df['Dept'].map({True: 'CSE', False: 'ECE'})
df.to_csv(file_name, index=False)
